In [269]:
import os
import json
import re
import spacy
import random
from typing import Optional
from spacy.training import Example
from datetime import datetime

In [270]:
##############################################################################
# 1. Funciones de limpieza y normalización (OCR issues)
##############################################################################
def fix_ocr_issues(text: str) -> str:
    correcciones = {
        "NACiÓN": "NACIÓN",
        "Colr,mbiana": "Colombiana",
        "Perscnal": "Personal",
        "AérEa": "Aérea",
        "No1956": "No 1956",
        "No.1956": "No. 1956",
        "N°1956": "N° 1956",
        "LEYNo": "LEY No",
        "LEYNo.": "LEY No.",
        "4JUN2019": "4 JUN 2019",
        "Dadaen": "Dada en",
        "Nc.": "No.",  # Agregamos esta corrección
        "2374'": "2374" #Agregamos esta correccion.
    }
    for error, correccion in correcciones.items():
        text = text.replace(error, correccion)

    # Mantener saltos de línea DOBLES, pero eliminar los sencillos DENTRO de párrafos.
    text = re.sub(r"(?<=\S)\n(?=\S)", " ", text)  # Junta líneas CON texto
    text = re.sub(r"\n{2,}", "\n\n", text)        # Máximo 2 saltos seguidos
    text = re.sub(r"[ \t]+", " ", text)  # Espacios/tabs a 1 espacio
    text = text.strip()
    text = re.sub(r"(No|N°|No\.)(\d+)", r"\1 \2", text, flags=re.IGNORECASE)
    text = re.sub(r"(\d+)([A-Z]{1,3}\d{4})", r"\1 \2", text)

    return text

In [271]:
##############################################################################
# 2. Regex para fechas, ley, etc. (ejemplo)
##############################################################################
def parse_date_str(text: str) -> Optional[str]:
    patterns = [
        r"(\d{1,2})\s*(?:de\s*)?([A-Z]+)\s*(?:de\s*)?(\d{4})",
        r"(\d{1,2})\s+([A-Z]{3,})\s+(\d{4})",
    ]
    month_map = {
        "ENE": "01", "FEB": "02", "MAR": "03", "ABR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AGO": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DIC": "12",
        "ENERO": "01", "FEBRERO": "02", "MARZO": "03", "ABRIL": "04",
        "MAYO": "05", "JUNIO": "06", "JULIO": "07", "AGOSTO": "08",
        "SEPTIEMBRE": "09", "OCTUBRE": "10", "NOVIEMBRE": "11", "DICIEMBRE": "12"
    }

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                day = match.group(1).zfill(2)
                month_str = match.group(2).upper()
                year = match.group(3)
                month = month_map.get(month_str)
                if month:
                    return f"{day}/{month}/{year}"
            except ValueError:
                continue
    return None



In [272]:
def parse_ley_and_type(text: str):
    type_map = {
        "orgánica": "orgánica",
        "organica": "orgánica",
        "ordinaria": "ordinaria",
        "estatutaria": "estatutaria",
        "estaturaria": "estatutaria",  # Podría haber errores de OCR
    }

    pattern = r"""
        (?:LEY|LEYES)[\s\n]*  # "LEY" o "LEYES", seguido de espacios/saltos
        (?:             # Grupo opcional para el tipo de ley
            (ESTATUTARIA|ORDINARIA|ORG[AÁ]NICA)  # Tipos de ley
            [\s\n]*     # 1 o más espacios/saltos DESPUÉS del tipo
        )?              # El tipo de ley es OPCIONAL
        (?:N[°ºc]?\.?|No\.?)?  # "N", "No", "N°", "Nº", "No.", "N.", "Nc.", opcional
        [\s\n]*         # 0 o más espacios/saltos
        (\d+)          # Captura el número de la ley
        (?:['´`‘’])?   # Caracteres especiales opcionales después del número
    """
    match = re.search(pattern, text, re.IGNORECASE | re.VERBOSE | re.DOTALL)

    if not match:
        return (None, None)  # Si no hay coincidencia, devuelve (None, None)

    raw_type = match.group(1)  # Captura el grupo 1 (tipo de ley, si existe)
    ley_number = match.group(2)  # Captura el grupo 2 (número de ley)

    if raw_type:
        tipo_ley = type_map.get(raw_type.lower()) #Uso del diccionario
    else:
        tipo_ley = "orgánica"  # Si no hay tipo explícito, asume "orgánica"

    return (tipo_ley, ley_number)

In [273]:
##############################################################################
# 3. Funciones de validación mínima (opcional)
##############################################################################
def looks_like_number_ley(value: str) -> bool:
    return bool(re.search(r"\d+", value)) if value else False

def looks_like_date(candidate: str) -> bool:
    return parse_date_str(candidate) is not None

def looks_like_epigrafe(text_epigrafe: str) -> bool:
    if not text_epigrafe:
        return False
    if len(text_epigrafe) < 30:
        return False
    if "POR MEDIO DE" not in text_epigrafe.upper():
        return False
    return True

In [274]:
def extract_first_section(text: str) -> str:
    """
    Devuelve el contenido desde el inicio del texto
    hasta la frase 'EL CONGRESO...' o 'EL CONGRESO DE LA REPUBLICA...'.
    Si no la encuentra, devuelve el texto completo.
    """
    pattern = r"^(.*?)(?=EL\s+CONGRESO\s+(?:DE\s+LA\s+REPUBLICA|DE\s+COLOMBIA))"
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1)
    return text


In [275]:
def extract_final_section(text: str) -> str:
    """
    Devuelve el contenido desde 'Dada, a los' (o una variante)
    hasta el final del texto. Si no lo encuentra, retorna cadena vacía.
    """
    pattern = r"(?:Dada,\s+a\s+los\s+.*)$"
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(0)
    return ""

In [276]:
def extract_final_date(text: str) -> str | None:
    """
    Extrae la fecha de la sección final del documento ("Dada, a los...").
    Devuelve la fecha normalizada (DD/MM/YYYY) o None si no la encuentra.
    """
    # Modificación: Ahora solo busca la fecha
    pattern = r"Dada,?\s+a\s+los\s+(.+)"  # Busca "Dada, a los" y captura lo que sigue
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        date_str = match.group(1).strip()  # Obtiene la parte de la fecha
        return parse_date_str(date_str)  # Usa la función de parseo para normalizar
    return None

In [277]:
def extraer_epigrafe(text: str, fecha_normalizada: str) -> str | None:
    if fecha_normalizada:
        try:
            datetime.strptime(fecha_normalizada, "%d/%m/%Y")
            start_index = text.find(fecha_normalizada)
            if start_index == -1:
                start_index = 0
                pattern_fecha = r"(\d{1,2})\s*([A-Za-z]{3,})\s*(\d{4})"
                match = re.search(pattern_fecha,text)
                if match:
                  start_index = match.end()
            else:
                start_index += len(fecha_normalizada)
        except ValueError:
           start_index = 0
           pattern_fecha = r"(\d{1,2})\s*([A-Za-z]{3,})\s*(\d{4})"
           match = re.search(pattern_fecha,text)
           if match:
              start_index = match.end()
    else:
        ley_match = re.search(r"(?:LEY|LEYES)[\s\n]*(?:N[°ºc]?\.?|No\.?)?[\s\n]*\d+", text, re.IGNORECASE)
        if ley_match:
            start_index = ley_match.end()
            mayuscula_match = re.search(r"[A-Z]", text[start_index:])
            if mayuscula_match:
                start_index += mayuscula_match.start()
        else:
            start_index = 0

    # Intenta buscar "EL CONGRESO" (con variaciones) o "REPUBLICA DE COLOMBIA"
    congreso_match = re.search(r"EL\s+CONGRESO|REPUBLICA\s+DE\s+COLOMBIA\s*-\s*GOBIERNO\s+NACIONAL", text, re.IGNORECASE)
    if congreso_match:
        end_index = congreso_match.start()
    else:
        end_index = len(text)

    if start_index < end_index:
        epigrafe = text[start_index:end_index].strip()
        # Priorizar la frase "POR"
        pattern_epigrafe = r"(POR\s+(?:MEDIO\s+DE\s+LA\s+CUAL|LA\s+CUAL)\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO|REPUBLICA\s+DE\s+COLOMBIA)"
        match_ep = re.search(pattern_epigrafe, epigrafe, re.IGNORECASE | re.DOTALL)

        if match_ep:  #Si encuentra un match con la frase
            return match_ep.group(1).strip()
        else: #Si no encuentra con la frase, intenta dentro de comillas
          pattern_epigrafe = r'["“”](.*?)["“”]'
          match_ep = re.search(pattern_epigrafe, epigrafe, re.DOTALL)
          if match_ep:
            return match_ep.group(1).strip()
          else: #Si no, retorna el texto entre los indices.
            return epigrafe
    return None

In [278]:
##############################################################################
# 5. spaCy: extraer metadatos (NUMERO_LEY, FECHA, EPIGRAFE)
##############################################################################
def spacy_extract_metadata(text: str, nlp) -> dict:
    doc = nlp(text)
    result = {
        "numero_ley": None,
        "fecha": None,
        "epigrafe": None
    }
    for ent in doc.ents:
        if ent.label_ == "NUMERO_LEY":
            result["numero_ley"] = ent.text
        elif ent.label_ == "FECHA":
            result["fecha"] = ent.text
        elif ent.label_ == "EPIGRAFE":
            result["epigrafe"] = ent.text
    return result


def normalize_spacy_date(spacy_fecha: Optional[str]) -> Optional[str]:
    if not spacy_fecha:
        return None
    parsed = parse_date_str(spacy_fecha)
    return parsed

In [279]:
##############################################################################
# 6. Función híbrida: extrae metadatos de un texto (regex + spaCy)
##############################################################################
def extract_metadata_layout_hybrid(text_lineal: str, nlp) -> dict:
    # 1) Recortar a la primera sección
    first_section = extract_first_section(text_lineal)

    # 2) Limpieza OCR, etc.
    first_section = fix_ocr_issues(first_section)

    # 3) Extraer con spaCy (por si detecta NUMERO_LEY, FECHA, EPIGRAFE)
    spacy_result = spacy_extract_metadata(first_section, nlp)

    # 4) Regex
    tipo_ley, ley_number = parse_ley_and_type(first_section)
    fecha_regex = parse_date_str(first_section)

    # 5) Normalizar fecha de spaCy
    spacy_fecha = spacy_result["fecha"]
    spacy_fecha_normalizada = normalize_spacy_date(spacy_fecha)
    if spacy_fecha_normalizada:
        final_fecha = spacy_fecha_normalizada
    else:
        final_fecha = fecha_regex

    # 6) Epígrafe
    epigrafe_regex = extraer_epigrafe(first_section, final_fecha)

    # 7) Fallback en número de ley
    spacy_numero_ley = spacy_result["numero_ley"]
    if not spacy_numero_ley or not looks_like_number_ley(spacy_numero_ley):
        spacy_numero_ley = ley_number

    # 8) Fallback epígrafe
    spacy_epigrafe = spacy_result["epigrafe"]
    if spacy_epigrafe:
        # si deseas validarlo con looks_like_epigrafe, hazlo
        final_epigrafe = spacy_epigrafe
    else:
        final_epigrafe = epigrafe_regex

    return {
        "numero_ley": spacy_numero_ley,
        "fecha": final_fecha,
        "tipo_ley": tipo_ley,
        "epigrafe": final_epigrafe
    }

In [280]:
def extract_metadata_two_phases(full_text: str, nlp) -> dict:
    # 1) Sección de arriba
    top_section = extract_first_section(full_text)
    top_metadata = extract_metadata_layout_hybrid(top_section, nlp)

    # 2) Sección de abajo (buscando 'Dada, a los')
    bottom_section = extract_final_section(full_text)
    bottom_metadata = extract_metadata_layout_hybrid(bottom_section, nlp)

    # 3) Combinar
    final_metadata = {
        "numero_ley": None,
        "fecha": None,
        "tipo_ley": None,
        "epigrafe": None
    }

    # Priorizar lo que venga en top_metadata
    final_metadata["numero_ley"] = top_metadata["numero_ley"] or bottom_metadata["numero_ley"]
    final_metadata["fecha"]      = top_metadata["fecha"]      or bottom_metadata["fecha"]
    final_metadata["tipo_ley"]   = top_metadata["tipo_ley"]   or bottom_metadata["tipo_ley"]

    # Epígrafe: si top_metadata lo trae, usarlo; si no, usar bottom
    #Ya no se prioriza el mas largo, sino que se toma el de la seccion inicial.
    final_metadata["epigrafe"] = top_metadata["epigrafe"] or bottom_metadata["epigrafe"]

    return final_metadata

In [281]:
##############################################################################
# 7. Función para procesar un archivo .txt completo (como si fuera "una sola página")
##############################################################################
def extract_metadata_from_txt_file(txt_path: str, nlp) -> dict:
    # Leer el contenido
    with open(txt_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Llamar la función de dos fases
    final_metadata = extract_metadata_two_phases(content, nlp)
    return final_metadata

In [282]:
##############################################################################
# 8. Función para procesar TODOS los .txt de una carpeta
##############################################################################
def procesar_txt_en_carpeta(carpeta_txt, carpeta_salida, nlp):
    os.makedirs(carpeta_salida, exist_ok=True)

    for archivo in os.listdir(carpeta_txt):
        if archivo.lower().endswith(".txt"):
            ruta_txt = os.path.join(carpeta_txt, archivo)

            metadatos = extract_metadata_from_txt_file(ruta_txt, nlp)

            # Guardar JSON
            nombre_json = os.path.splitext(archivo)[0] + ".json"
            ruta_json = os.path.join(carpeta_salida, nombre_json)
            with open(ruta_json, "w", encoding="utf-8") as f:
                json.dump(metadatos, f, ensure_ascii=False, indent=4)

            print(f"Procesado {archivo} -> Metadatos en: {ruta_json}")


In [283]:
##############################################################################
# 9. (Opcional) Entrenar y cargar modelo spaCy
##############################################################################
def train_spacy_model(train_data, output_dir="modelo_leyes_epigrafe"):
    nlp = spacy.blank("es")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

    # Añadir etiquetas
    for text, annotations in train_data:
        for start, end, label in annotations["entities"]:
            ner.add_label(label)

    nlp.initialize()
    examples = []
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)

    for i in range(10):
        random.shuffle(examples)
        losses = {}
        for example in examples:
            nlp.update([example], losses=losses)
        print(f"Época {i}, pérdidas: {losses}")

    nlp.to_disk(output_dir)
    print(f"Modelo guardado en: {output_dir}")
    return nlp

def load_spacy_model(model_path="modelo_leyes_epigrafe"):
    return spacy.load(model_path)

In [284]:
if __name__ == "__main__":
    # Rutas de carpeta
    carpeta_txt = r"c:/Users/Jorge/OneDrive/Documents/proyect/document/leyes"
    carpeta_salida = r"c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024"

    # Cargar tu modelo spaCy (si ya lo tienes entrenado)
    nlp_trained = load_spacy_model("modelo_leyes_epigrafe")  # ejemplo
    # O si estás usando un EntityRuler, define nlp_trained antes.

    # Procesar todos los .txt de la carpeta
    procesar_txt_en_carpeta(carpeta_txt, carpeta_salida, nlp_trained)

Procesado LEY-2350-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2350-2024plaintext.json
Procesado LEY-2351-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2351-2024plaintext.json
Procesado LEY-2352-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2352-2024plaintext.json
Procesado LEY-2353-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2353-2024plaintext.json
Procesado LEY-2354-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2354-2024plaintext.json
Procesado LEY-2355-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2355-2024plaintext.json
Procesado LEY-2356-2024plaintext.txt -> Metadatos en: c:/Users/Jorge/OneDrive/Documents/proyect/docu